# Uncertain Mortality Model vs American Options

#### Re-insurance Deals

In this notebook, we study the *Uncertain Mortality Model* for the pricing of reinsurance deals. For the sake of simplicity, we only consider one type of default: the risk of death. (lapse is ignored)

We consider the following reinsurance product:

- At maturity, if the insurance subscriber is alive, the issuer delivers a put on the underlying $X$

$$
u^\textrm{mat}( x) = (K_\textrm{mat} - x)_+.
$$

- At the time of death, if it is before the maturity, the issuer delivers an exit payoff, typically another put on the underlying $X$.

$$
u^D(t, x) = (K_D - x)_+.
$$

- The subscriber pays a constant fee $α\Delta t$ at every time step until death or the maturity or the product

- The insurance sells a large number of these contracts to subscribers. We assume that the times of death of the subscribers $\tau^D$ are independent, and identically distributed, and also independent of the underlying's stock price.

- We assume that the underlying's risk neutral price dynamics is the Black-Scholes model with zero interest rate/repo/dividend yield

$$
dX_t = \sigma X_t d W_t.
$$

#### The Insurers' Approach and Risk-Neutral Pricing

This contract shows two types of risk: the times of death of the subscribers and the changes in the price of the underlying. 

In this case, the issuer can apply the insurer's approach to the risk of death times, i.e., the law of large numbers. The more people buy the contract, the less risk.

Choosing a risk-neutral measure under which the death times $\tau^D$ have the same distribution as under the historical probability measure is equivalent to applying the arbitrage-pricing approach to the financial risk insurer's rule on the risk of death. The price of the contract is then

$$
u(t, x) = \mathbb{E}^\mathbb{Q}_{t, x} \left[u^\textrm{mat}(X_T) \mathbb{1}_{\tau^D \geq T} -\int_t^T\alpha\mathbb{1}_{\tau^D>s}ds+ u^D(\tau^D, X_{\tau^D}) \mathbb{1}_{\tau^D < T} \right].
$$

#### Deterministic Death Rate

If the death intensity is a deterministic function $\lambda_t^D$ (i.e. $\tau^D$ has an exponential distribution with time-dependent intensity $\lambda_t^D$), then we have seen that $u$ is the solution to the linear PDE

$$
\left\{\begin{array}{l}
\partial_t u + \frac{1}{2} \sigma^2 x^2 \partial_x^2 u -\alpha + \lambda_t^D \cdot (u^D - u) = 0,\\
u(T, \cdot) \equiv u^\textrm{mat}.
\end{array}\right.
$$

#### Uncertain Mortality Model

If the death rate is uncertain, we assume that it is adapted (i.e., it does not look into the future) and belongs to a moving corridor
$\left[\underline{\lambda}_t, \overline{\lambda}_t\right]$. The most conservative way to price the contract is to compute the (financially) worst death rate process $\lambda_t^D$ as being chosen so as to maximize the value of the contract. The resulting HJB equation is

$$
\left\{\begin{array}{l}
\partial_t u + \frac{1}{2} \sigma^2 x^2 \partial^2_x u -\alpha + \Lambda^D(t, u^D - u) \cdot (u^D - u) = 0,\\
u(T, \cdot) \equiv u^\textrm{mat},
\end{array}\right.
$$
where the function $\Lambda^D$ is defined by
$$
\Lambda^D (t, y) = \left\{\begin{array}{l}
    \overline{\lambda}^D_t \quad \textrm{if} \ y \geq 0, \\
    \underline{\lambda}^D_t \quad \textrm{otherwise.}
\end{array}\right.
$$

## Link with $1$-BSDE and Numerical Schemes

From the Pardoux-Peng theorem, we know that the solution $u(0, x)$ can be represented as the solution $Y_0^x$ to the $1$-BSDE

$$
dY_t = -f(t, X_t, Y_t, Z_t) \, dt + Z_t \, dW_t,
$$

with the terminal condition $Y_T = u^\textrm{mat} (X_T)$, where $X_0 = x$ and

$$
f(t, x, y, z) = -\alpha + \Lambda^D(t, u^D(t, x) - y) \cdot (u^D(t, x) - y).
$$


## BSDE Discretization

### Explicit Euler Schemes

\begin{align*}
Y_{t_n} =& \ u^{\text{mat}}\left(X_{t_n}\right)\\
Y_{t_{i - 1}} =& \ \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] -\alpha\Delta t_i + \Lambda^D \left(t_{i - 1}, u^D(t_{i - 1} , X_{t_{i - 1}}) - \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ]\right) \cdot \left( u^D(t_{i - 1}, X_{t_{i - 1}}) - \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] \right) \Delta t_i\\
=& -\alpha\Delta t_i + \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] \left( 1 - \overline{\lambda}^D \Delta t_i \right) +
u^D(t_{i - 1}, X_{t_{i - 1}}) \overline{\lambda}^D \Delta t_i \right) \mathbb{1}_{u^D\left(t_{i - 1}, X_{t_{i - 1}}\right) \geq \mathbb{E}_{i-1}[Y_{t_i}]}\\
& + \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] \left( 1 - \underline{\lambda}^D \Delta t_i \right) +
u^D(t_{i - 1}, X_{t_{i - 1}}) \underline{\lambda}^D \Delta t_i \right) \mathbb{1}_{u^D\left(t_{i - 1}, X_{t_{i - 1}}\right) < \mathbb{E}_{i-1}[Y_{t_i}]}.
\end{align*}

### Implicit Euler Scheme

\begin{align*}
Y_{t_n} =& \ u^{\text{mat}}\left(X_{t_n}\right)\\
Y_{t_{i - 1}} =& \ \mathbb{E}^\mathbb{Q}_{i - 1} [Y_{t_i}] -\alpha\Delta t_i+ \Lambda^D \left(t_{i - 1}, u^D(t_{i - 1}, X_{t_{i - 1}}) - Y_{t_{i - 1}}\right) \cdot \left( u^D(t_{i - 1}, X_{t_{i - 1}}) - Y_{t_{i - 1}} \right) \Delta t_i.
\end{align*}

The implicit scheme involves $Y_{t_{i - 1}}$ on both sides of the equation, which generally requires a root-finding routine to find $Y_{t_{i - 1}}$. However in this specific case, it can be shown that 

\begin{equation}u^D(t_{i-1}, X_{t_{i-1}}) \geq Y_{t_{i-1}}\quad\text{if and only if}\quad u^D(t_{i-1}, X_{t_{i-1}}) \geq \mathbb{E}^{\mathbb{Q}}_{i-1}\left[Y_{t_i}\right]-\alpha\Delta t_i.\end{equation}

Thus

\begin{split}
Y_{t_{i - 1}} =& \frac{1}{1 + \overline{\lambda}^D \Delta t_i} \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] - \alpha\Delta t_i +
u^D(t_{i - 1}, X_{t_{i - 1}}) \overline{\lambda}^D \Delta t_i \right) \mathbb{1}_{u^D\left(t_{i - 1}, X_{t_{i - 1}}\right) \geq \mathbb{E}_{i-1}[Y_{t_i}]-\alpha\Delta t_i}\\
& + \frac{1}{1 + \underline{\lambda}^D \Delta t_i} \left( \mathbb{E}^\mathbb{Q}_{i - 1} [ Y_{t_i} ] -\alpha\Delta t_i +
u^D(t_{i - 1}, X_{t_{i - 1}}) \underline{\lambda}^D \Delta t_i \right) \mathbb{1}_{u^D(t_{i - 1}, X_{t_{i - 1}}) < \mathbb{E}_{i-1}[Y_{t_i}]-\alpha\Delta t_i}.
\end{split}



In [29]:
## Imports 

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import warnings
from scipy.stats import norm
from collections import deque
warnings.filterwarnings("ignore")

In [30]:
def blackscholes_mc(S=100, vol=0.2, r=0, q=0, ts=np.linspace(0, 1, 13), npaths=10):
    """Generate Monte-Carlo paths in Black-Scholes model.

    Parameters
    ----------
    S: scalar
        The spot price of the underlying security.
    vol: scalar
        The implied Black-Scholes volatility.
    r: scalar
        The annualized risk-free interest rate, continuously compounded.
    q: scalar
        The annualized continuous dividend yield.
    ts: array_like
        The time steps of the simualtion
    npaths: int
        the number of paths to simulate

    Returns
    -------
    paths: ndarray
        The Monte-Carlo paths.
    """
    nsteps = len(ts) - 1
    ts = np.asfarray(ts)[:, np.newaxis]
    W = np.cumsum(np.vstack((np.zeros((1, npaths), dtype=np.float),
                             np.random.randn(nsteps, npaths) * np.sqrt(np.diff(ts, axis=0)))),
                  axis=0)
    paths = np.exp(-0.5*vol**2*ts + vol*W)*S*np.exp((r-q)*ts)
    return paths

def blackscholes_price(K, T, S, vol, r=0, q=0, callput='call'):
    """Compute the call/put option price in the Black-Scholes model
    
    Parameters
    ----------
    K: scalar or array_like
        The strike of the option.
    T: scalar or array_like
        The maturity of the option, expressed in years (e.g. 0.25 for 3-month and 2 for 2 years)
    S: scalar or array_like
        The current price of the underlying asset.
    vol: scalar or array_like
        The implied Black-Scholes volatility.
    r: scalar or array_like
        The annualized risk-free interest rate, continuously compounded.
    q: scalar or array_like
        The annualized continuous dividend yield.
    callput: str
        Must be either 'call' or 'put'.

    Returns
    -------
    price: scalar or array_like
        The price of the option.

    Examples
    --------
    >>> blackscholes_price(95, 0.25, 100, 0.2, r=0.05, callput='put')
    1.5342604771222823
    """
    F = S*np.exp((r-q)*T)
    v = np.sqrt(vol**2*T)
    d1 = np.log(F/K)/v + 0.5*v
    d2 = d1 - v
    try:
        opttype = {'call':1, 'put':-1}[callput.lower()]
    except:
        raise ValueError('The value of callput must be either "call" or "put".')
    price = opttype*(F*norm.cdf(opttype*d1)-K*norm.cdf(opttype*d2))*np.exp(-r*T)
    return price


def pwlinear_basis(xknots):
    """Basis that represent a piecewise linear function with given knots"""
    fs = [lambda x: np.ones_like(x, dtype=np.float), lambda x: x-xknots[0]]
    fs.extend([lambda x, a=xknots[i]: np.maximum(x-a, 0) for i in range(len(xknots))])
    return fs


def pwlinear_fit(xdata, ydata, xknots):
    """Fit a piecewise linear function with xknots to xdata and ydata"""
    fs = pwlinear_basis(xknots)
    A = np.vstack([f(xdata) for f in fs]).T
    ps = np.linalg.lstsq(A, ydata, rcond=None)[0]
    return ps, fs

To be more specific about the payouts of the product, we make the following assumptions:

- We assume that all transactions occur at the end of the month, i.e. the insurance subsriber is expected to pay a fee of $\alpha/12$ at the end of each month; if the subscriber dies in the middle of the month, the death payment will be made at the end of that month. 

- If the death occurs at the maturity, we assume that the death comes first and the death payment $u^D$ will be made.

<b style="color:darkorange"> Example 1 : </b> (Deterministic Mortality Rate Model)

We assume that the underlying's risk neutral price dynamics is the Black-Scholes model with zero interest rate/repo/dividend yield

$$dX_t = \sigma X_t d W_t,\quad X_0=100,\quad\sigma=0.3$$

We shall price the reinsurance deal using two Monte Carlo methods explained below :

In [31]:
# note that lambda is a reserved keyword in python, so we use lambd instead
def exponential_samples(n, lambd):
    """Generate n samples from exponential distribution with rate lambd"""
    return -np.log(np.random.rand(n))/lambd

<b>(a).</b> (Direct simulation of death times) Monte Carlo simulation by directly simulating death time $\tau^D$ for each path to estimate the quantity

$$u(t, x) = \mathbb{E}^\mathbb{Q}_{t, x} \left[u^\textrm{mat}(X_T) \mathbb{1}_{\tau^D \geq T} - \int_t^T\alpha \mathbb{1}_{\tau^D> s}ds
+ u^D(\tau^D, X_{\tau^D}) \mathbb{1}_{\tau^D < T} \right],$$

when $T=10$, $K_{\text{mat}}=90$, $K_D=100$, $\alpha=3$, and $\tau^D$ has an exponential distribution of constant intensity $\lambda^D=0.025$

In [26]:
S = 100
vol = 0.3
T = 10
ts = np.linspace(0, T, int(np.round(T*12))+1)
alpha = 3
lambda_d = 0.025
K_mat = 90.0
K_D = 100.0
u_mat = lambda x : np.maximum(K_mat - x, 0)
u_D   = lambda x : np.maximum(K_D - x, 0) 

In [5]:
npaths = 100000
np.random.seed(2954)
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
tau_d = exponential_samples(npaths,lambda_d)

In [6]:
ind,_ind = np.where(tau_d < T)[0],np.where(tau_d >= T)[0]
payoff_D = np.array([u_D(paths[np.argmax(ts>tau_d[i])][i]) - alpha*ts[np.argmax(ts>tau_d[i])] for i in ind])
payoff_mat = u_mat(paths[-1][_ind])-alpha*T
fairPrice = (np.sum(payoff_D) + np.sum(payoff_mat))/npaths
fairPrice

2.1197249917261867

<b>(b).</b> (Averaing over death events) Using the Feynman-Kac formula, a stochastic representation of the linear PDE 

$$\left\{\begin{array}{l}\partial_t u + \frac{1}{2}\sigma^2 x^2 \partial_x^2 u - \alpha + \lambda^D \cdot (u^D - u) = 0\\
        u(T, x) \equiv u^\textrm{mat}(x)\end{array}\right.$$
        
that does not involve simulating death times $\tau^D$ 

In [41]:
Cp = np.zeros_like(paths)
Cp[-1] = u_mat(paths[-1])
dt = 1/12
for t in range(len(paths)-2,-1,-1) :
    Cp[t] = np.exp(-lambda_d*dt)*Cp[t+1] - alpha*dt + (1-np.exp(-lambda_d*dt))*u_D(paths[t+1])
    
fairPrice = np.mean(Cp[0])
print(fairPrice)

2.1077793704462353


$\text{Explanation of how we arrive at the stochastic representation :}$

Consider $u \in C^{1,2}\left([0, T) \times \mathbb{R}^{d}\right) \cap C^{0}\left([0, T] \times \mathbb{R}^{d}\right)$ with quadratic growth in $x$ uniformly in $t$ and solution to the parabolic PDE:
$$
\partial_{t} u(t, x)+\mathcal{L} u(t, x)-r(t, x) u(t, x)+f(t, x)=0
$$
with terminal condition $u(T, x)=g(x)$. Then $u$ admits the stochastic representation

$$u(t, x) = \mathbb{E}^\mathbb{Q}_{t, x} \left[u^\textrm{mat}(X_T) \mathbb{1}_{\tau^D \geq T} - \int_t^T\alpha \mathbb{1}_{\tau^D> s}ds
+ u^D(\tau^D, X_{\tau^D}) \mathbb{1}_{\tau^D < T} \right],$$

$$ \mathbb{E}^\mathbb{Q}_{t, x} \left[u^\textrm{mat}(X_T) \mathbb{1}_{\tau^D \geq T}\right] = e^{-\int_{t}^{T} λ^D _s ds }\mathbb{E}^\mathbb{Q}_{t, x}\left[u^{mat}(X_T)\right] 
$$

$$ 
\mathbb{E}^\mathbb{Q}_{t, x}\left[- ∫_{t}^{T} α1_{τ^D \geq s}ds \right] = -α∫_{t}^{T} e^{-\int_{t}^{s} λ_{v}^{D}dv}
$$

$$ 
\mathbb{E}^\mathbb{Q}_{t, x}\left[u^D(τ^D,X_{τ^D})1_{τ^D < T} \right] = \mathbb{E}^\mathbb{Q}_{t, x}\left[ \int_{t}^{T} u^D(s,X_s)λ_s^D e^{-\int_t^sλ_v^D}\right]
$$

$$
\textrm{where} \ \mathbb{E}_{t, x}\left[.\right] = \mathbb{E}\left[.|X_t = x,τ>t\right]   
$$

we also have

$$
u(t, x)=\mathbb{E}^{\mathbb{Q}}_{t, x}\left[\int_{t}^{T} e^{-\int_{t}^{s} r\left(u, X_{u}\right) d u} f\left(s, X_{s}\right) d s+e^{-\int_{t}^{T} r\left(s, X_{s}\right) d s} g\left(X_{T}\right) \mid X_{t}=x\right]
$$
With f as an intermediate payoff and the Ito generator as $\mathcal{L}=\frac{1}{2} \sigma^2 x^2 \partial_{x}^2$ here, in the Feynman-Kac formula, we put $r(t,x) = \lambda^D$ and $f(t,x)= \lambda^D u^D(t,x) - \alpha$, $g=u_{mat}$ 

$$
u(t, x)=\mathbb{E}^{\mathbb{Q}}\left[\int_{t}^{T} e^{-\lambda^D \times (s-t)} \left(\lambda^D u^D(t,x)- \alpha \right) d s+e^{-\lambda^D \times (T-t)} u_{mat}(X_{T}) \mid X_{t}=x\right]
$$

we get
<br>

  \begin{array}{l}\partial_t u + \frac{1}{2}\sigma^2 x^2 \partial_x^2 u - \alpha + \lambda^D \cdot (u^D - u) = 0\\
        u(T, x) \equiv u^\textrm{mat}(x)\end{array}

----------------------------------------------------

<b style="color:darkorange">Example 2 : </b> (Uncertain Mortality Rate Model - BSDE)

We assume that $\lambda^D_t \in \left[\underline{\lambda}^D, \overline{\lambda}^D \right]$, where $\underline{\lambda}^D=0.005$ and $\overline{\lambda}^D=0.04$ 


<b>(a)</b> We use the implicit Euler scheme for BSDE. Using piecewise-linear fit with 10 knots to estimate the conditional expectation $\mathbb{E}^{\mathbb{Q}}_{i-1}\left[Y_{t_i}\right]$

In [45]:
lambda_max = 0.04
lambda_min = 0.005
LAMBDA_d = lambda x : lambda_max if x>=0 else lambda_min
LAMBDA_d = np.vectorize(LAMBDA_d)
q = deque()

Yt = np.zeros_like(paths)
Yt[-1] = u_mat(paths[-1])
dt = 1/12

for t in range(len(paths)-2,0,-1) :
    xknots = np.linspace(np.percentile(paths[t], 1), np.percentile(paths[t], 99), 10)
    ps, fs = pwlinear_fit(paths[t], Yt[t+1], xknots)
    q.append((ps,fs))
    EYt    = sum([f(paths[t])*p for (f, p) in zip(fs, ps)])
    y = u_D(paths[t])-(EYt-alpha*dt)
    Yt[t] = (EYt - alpha*dt + LAMBDA_d(y)*u_D(paths[t])*dt)/(1+LAMBDA_d(y)*dt)
        
    
fairPrice = np.mean(Yt[1]-alpha*dt)
print(fairPrice)

3.315064524680559


<b>(b)</b> Running an independent new simuation using the estimates of $\mathbb{E}^{\mathbb{Q}}_{i-1}\left[Y_{t_i}\right]$ obtained in (a) to get a lower bound estimate.

In [46]:
### Independent Simulation 
ind_paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
Yt = np.zeros_like(ind_paths)
Yt[-1] = u_mat(ind_paths[-1])
for t in range(len(ind_paths)-2,0,-1) :
    ps,fs = q.popleft()
    EYt    = sum([f(ind_paths[t])*p for (f, p) in zip(fs, ps)])
    y = u_D(ind_paths[t])-(EYt-alpha*dt)
    Yt[t] = EYt - alpha*dt + LAMBDA_d(y)*y*dt
    
fairPrice = np.mean(Yt[1]-alpha*dt)
print(fairPrice)


3.3073301734464806


<b>(c).</b> Comparing the price in the model with deterministic mortality rate $\underline{\lambda}^D$ and with deterministic mortality rate $\overline{\lambda}^D$

with deterministic mortality rate $\underline{\lambda}^D$

In [47]:
lambda_d = 0.005
Cp = np.zeros_like(paths)
Cp[-1] = u_mat(paths[-1])
dt = 1/12
for t in range(len(paths)-2,-1,-1) :
    Cp[t] = np.exp(-lambda_d*dt)*Cp[t+1] - alpha*dt + (1-np.exp(-lambda_d*dt))*u_D(paths[t+1])
    
fairPrice = np.mean(Cp[0])
print(fairPrice)

0.40906682038746267


with deterministic mortality rate $\overline{\lambda}^D$

In [48]:
lambda_d = 0.04
Cp = np.zeros_like(paths)
Cp[-1] = u_mat(paths[-1])
dt = 1/12
for t in range(len(paths)-2,-1,-1) :
    Cp[t] = np.exp(-lambda_d*dt)*Cp[t+1] - alpha*dt + (1-np.exp(-lambda_d*dt))*u_D(paths[t+1])
    
fairPrice = np.mean(Cp[0])
print(fairPrice)

3.179281680794135


-------------------------------------------------------------------------------------------------------------------------------

<b style="color:darkorange">Example 3 :</b> (Uncertain Mortality Rate Model - Longstaff and Schwartz) In addition to the BSDE simulation, the reinsurance deals can be priced by adapting the Longstaff-Schwartz algorithm
The same lower and upper bounds of $\lambda_t^D$ are used. 



In [49]:
# Longstaff-Schwartz algorithm for Uncertain Mortality Rate Model
S = 100
vol = 0.3
T = 10
ts = np.linspace(0, T, int(np.round(T*12))+1)
alpha = 3
lambda_min = 0.005
lambda_max = 0.04

K_mat = 90.0
K_D = 100.0

def u_mat(x):
    return np.maximum(K_mat - x, 0)

def u_D(x):
    return np.maximum(K_D - x, 0)


# first Monte Carlo run to estimate the value function by regressions
npaths = 5000
betas = np.zeros((len(ts), 2), dtype=np.float)
V = u_mat(paths[-1])
lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
K = 0.5*(K_mat+K_D)
for i in range(len(ts)-2, 0, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
    A = np.vstack((np.ones_like(Z), Z)).T
    betas[i] = np.linalg.lstsq(A, V, rcond=None)[0]
    lambd = np.where(u_D(paths[i]) >= betas[i, 0]+betas[i, 1]*Z, lambda_max, lambda_min)

# independent simulation to obtain a lower bound
npaths = 100000
ind_paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
V = u_mat(ind_paths[-1])
lambd = np.where(u_D(ind_paths[-1]) >= V, lambda_max, lambda_min)
K = 0.5*(K_mat+K_D)
for i in range(len(ts)-2, -1, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(ind_paths[i+1]) + (1-p)*V - alpha*dt
    Z = blackscholes_price(K, ts[-1]-ts[i], ind_paths[i], vol, callput='put')
    lambd = np.where(u_D(ind_paths[i]) >= betas[i, 0]+betas[i, 1]*Z, lambda_max, lambda_min)
np.mean(V)

3.139202249155837

<b>(a).</b> In the above code, we use European put price (the strike is taken to be the average of $K_{\text{mat}}$ and $K_D$) and constant as basis functions to estimate the sum of future payoffs. We can modify the code by using the piecewise-linear fit to estimate the sum of future payoffs.

In [50]:
npaths = 5000
betas = np.zeros((len(ts), 2), dtype=np.float)
V = u_mat(paths[-1])
lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
K = 0.5*(K_mat+K_D)
q = deque()

for i in range(len(ts)-2, 0, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
    zknots = np.linspace(np.percentile(Z, 1), np.percentile(Z, 99), 20)
    ps, fs = pwlinear_fit(Z, V, zknots)
    q.append((ps,fs))
    EV    = sum([f(Z)*p for (f, p) in zip(fs, ps)])
#     A = np.vstack((np.ones_like(Z), Z)).T
#     betas[i] = np.linalg.lstsq(A, V, rcond=None)[0]
    lambd = np.where(u_D(paths[i]) >= EV, lambda_max, lambda_min)

# independent simulation to obtain a lower bound
npaths = 100000
ind_paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
V = u_mat(ind_paths[-1])
lambd = np.where(u_D(ind_paths[-1]) >= V, lambda_max, lambda_min)
K = 0.5*(K_mat+K_D)
for i in range(len(ts)-2, -1, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(ind_paths[i+1]) + (1-p)*V - alpha*dt
    if i!=0 :
        Z = blackscholes_price(K, ts[-1]-ts[i], ind_paths[i], vol, callput='put')
        ps,fs = q.popleft()
        EV    = sum([f(Z)*p for (f, p) in zip(fs, ps)])
        lambd = np.where(u_D(ind_paths[i]) >= EV, lambda_max, lambda_min)
np.mean(V)

3.203124235190343

<b>(b).</b> We compare the BSDE scheme with the Longstaff-Schwartz-like scheme in terms of variance. For each method we run the two-step procedure (regression and indedenpent pricing) 20 times to estimate variance

In [51]:
bsde_values = []
ls_values = []

for iteration in range(20) :
    npaths = 5000
    paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
    betas = np.zeros((len(ts), 2), dtype=np.float)
    V = u_mat(paths[-1])
    lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
    lambda_max = 0.04
    lambda_min = 0.005
    LAMBDA_d = lambda x : lambda_max if x>=0 else lambda_min
    LAMBDA_d = np.vectorize(LAMBDA_d)  
    K = 0.5*(K_mat+K_D)
    Yt = np.zeros_like(paths)
    Yt[-1] = u_mat(paths[-1])
    ls_q = deque()
    bsde_q = deque()

    for i in range(len(ts)-2, 0, -1):
        dt = ts[i+1]-ts[i]
        p = 1-np.exp(-lambd*dt)
        V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
        Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
        zknots = np.linspace(np.percentile(Z, 1), np.percentile(Z, 99), 20)
        ps, fs = pwlinear_fit(Z, V, zknots)
        ls_q.append((ps,fs))
        EV    = sum([f(Z)*p for (f, p) in zip(fs, ps)])
    #     A = np.vstack((np.ones_like(Z), Z)).T
    #     betas[i] = np.linalg.lstsq(A, V, rcond=None)[0]
        lambd = np.where(u_D(paths[i]) >= EV, lambda_max, lambda_min)

    for t in range(len(paths)-2,0,-1) :
        xknots = np.linspace(np.percentile(paths[t], 1), np.percentile(paths[t], 99), 10)
        ps, fs = pwlinear_fit(paths[t], Yt[t+1], xknots)
        bsde_q.append((ps,fs))
        EYt    = sum([f(paths[t])*p for (f, p) in zip(fs, ps)])
        y = u_D(paths[t])-(EYt-alpha*dt)
        Yt[t] = (EYt - alpha*dt + LAMBDA_d(y)*u_D(paths[t])*dt)/(1+LAMBDA_d(y)*dt)
        


    # independent simulation to obtain a lower bound
    npaths = 50000
    paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
    V = u_mat(paths[-1])
    lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
    K = 0.5*(K_mat+K_D)
    for i in range(len(ts)-2, -1, -1):  ## iteration for longstaff-schwartz algo
        dt = ts[i+1]-ts[i]
        p = 1-np.exp(-lambd*dt)
        V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
        if i!=0 :
            Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
            ps,fs = ls_q.popleft()
            EV    = sum([f(Z)*p for (f, p) in zip(fs, ps)])
            lambd = np.where(u_D(paths[i]) >= EV, lambda_max, lambda_min)
    ls_values.append(np.mean(V))

    Yt = np.zeros_like(paths)
    Yt[-1] = u_mat(paths[-1])
    for t in range(len(paths)-2,0,-1) :  ## iteration for bsde algo
        ps,fs = bsde_q.popleft()
        EYt    = sum([f(paths[t])*p for (f, p) in zip(fs, ps)])
        y = u_D(paths[t])-(EYt-alpha*dt)
        Yt[t] = EYt - alpha*dt + LAMBDA_d(y)*y*dt
        
    bsde_values.append(np.mean(Yt[1]-alpha*dt))


In [52]:
## Calculating Mean and Variance for both methods  
print("Longstff-Schwartz Algorithm -> Mean : ",np.mean(ls_values),"Variance : ",np.std(ls_values)**2)
print("BSDE Algorithm -> Mean : ",np.mean(bsde_values),"Variance : ",np.std(bsde_values)**2)


Longstff-Schwartz Algorithm -> Mean :  3.2294429939252227 Variance :  0.011764633785905572
BSDE Algorithm -> Mean :  3.2761140369045014 Variance :  0.07019215170355601


We observe that Longstaff-Schwartz Algorithm has lower variance as compared to BSDE Algorithm.  

<b>(c).</b> The value of the fee $\alpha$ that makes the deal costless at inception : 

In [53]:
## We first observe a positive price for alpha=3.3 and negative value for alpha=3.4
## Now we perform a binary search between 3.3 and 3.4 till we get fair price of 0 (approx. to 4 decimal places)
## We also chose two different seeds for independent simulation otherwise our data will keep changing and we will never be able to reach 0 value since the 
## fair value will also keep changing

value = 1
alpha_max = 3.4
alpha_min = 3.3

while value!=0 :
    alpha = (alpha_max+alpha_min)/2
    npaths = 5000
    np.random.seed(2954)
    paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
    betas = np.zeros((len(ts), 2), dtype=np.float)
    V = u_mat(paths[-1])
    lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
    K = 0.5*(K_mat+K_D)
    q = deque()

    for i in range(len(ts)-2, 0, -1):
        dt = ts[i+1]-ts[i]
        p = 1-np.exp(-lambd*dt)
        V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
        Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
        zknots = np.linspace(np.percentile(Z, 1), np.percentile(Z, 99), 20)
        ps, fs = pwlinear_fit(Z, V, zknots)
        q.append((ps,fs))
        EV    = sum([f(Z)*p for (f, p) in zip(fs, ps)])
        lambd = np.where(u_D(paths[i]) >= EV, lambda_max, lambda_min)



    # independent simulation to obtain a lower bound
    npaths = 100000
    np.random.seed(10)
    paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
    V = u_mat(paths[-1])
    lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
    K = 0.5*(K_mat+K_D)
    for i in range(len(ts)-2, -1, -1):
        dt = ts[i+1]-ts[i]
        p = 1-np.exp(-lambd*dt)
        V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
        if i!=0 :
            Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
            ps,fs = q.popleft()
            EV    = sum([f(Z)*p for (f, p) in zip(fs, ps)])
            lambd = np.where(u_D(paths[i]) >= EV, lambda_max, lambda_min)
    
    value = np.round(np.mean(V),4)
    
    if value>0 :
        alpha_min = alpha
    if value<0:
        alpha_max = alpha
    
print(alpha,value)    
    
    

3.3710205078125 0.0


A value of around 3.37 will give us 0 fair price of the contract

---

<b style="color:darkorange">Example 4 :</b> Setting $K_D=0$, and all the other parameters remain the smae, in particualr $\alpha=3$. Then the default event is equivalent to lapse (with no mortality payoff). It has been observed that insurance subscribers do not lapse optimally, which explains the use of the uncertain lapse model with minimum and maximum lapse rates $\underline{\lambda}$, $\overline{\lambda}$. 


 If insurance subscribers were to lapse optimally, i.e., exercise optimally their option to lapse, how would you price the contract? The insurance subscriber can always lapse. A rational policy holder should lapse as soon as the payoff from exercising is greater than continuing the policy. So this is similar to American option and we can price this as an american option. Here, lambda max accounts for the fact that policy holders may not lapse when they should and lambda min accounts for the fact that they may lapse when they should not. We can use lambda max = infinty and lambda min = 0 to price this as an american option.

In the uncertain lapse model we can use lambda max = infinity and lambda min = 0 to price this with optimal exercise. Here the payoff from lapse is 0 and at every time one also has to pay the fees. The values of lambda denote that the policy holder can lapse at any time.

In [32]:
npaths = 5000
np.random.seed(2954)
u_D = lambda x : 0
lambda_max = np.inf
lambda_min = 0
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
betas = np.zeros((len(ts), 2), dtype=np.float)
V = u_mat(paths[-1])
lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
K = K_mat
q = deque()

for i in range(len(ts)-2, 0, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
    Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
    zknots = np.linspace(np.percentile(Z, 1), np.percentile(Z, 99), 20)
    ps, fs = pwlinear_fit(Z, V, zknots)
    q.append((ps,fs))
    EV    = sum([f(Z)*p for (f, p) in zip(fs, ps)])
    lambd = np.where(u_D(paths[i]) >= EV, lambda_max, lambda_min)

    
    
# independent simulation to obtain a lower bound
npaths = 100000
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
V = u_mat(paths[-1])
lambd = np.where(u_D(paths[-1]) >= V, lambda_max, lambda_min)
K = K_mat
for i in range(len(ts)-2, -1, -1):
    dt = ts[i+1]-ts[i]
    p = 1-np.exp(-lambd*dt)
    V = p*u_D(paths[i+1]) + (1-p)*V - alpha*dt
    if i!=0 :
        Z = blackscholes_price(K, ts[-1]-ts[i], paths[i], vol, callput='put')
        ps,fs = q.popleft()
        EV    = sum([f(Z)*p for (f, p) in zip(fs, ps)])
        lambd = np.where(u_D(paths[i]) >= EV, lambda_max, lambda_min)
np.mean(V)

3.7868522177418984

Implementing Longstaff-Schwartz algorithm to price the American option with payoff $u^D=0$ (peforming a second indepedent run to get a clean lower bound price). We compare the price from uncertain lapse model with the American price.

In [42]:
np.random.seed(2954)
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
payoff = np.maximum(K_mat-paths[-1], 0)-alpha*T
dt = 1/12
q = deque()

for i in range(len(ts)-2, 0, -1):
    p = np.polyfit(paths[i], payoff, deg=2)
    q.append(p)
    contval = np.polyval(p, paths[i])
    exerval = np.zeros(len(paths[i]))-alpha*ts[i]
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]
    
np.mean(payoff)
    
## Independent Monte Carlo Sumulation
paths = blackscholes_mc(S=S, vol=vol, r=0, q=0, ts=ts, npaths=npaths)
payoff = np.maximum(K_mat-paths[-1], 0)-alpha*T
for i in range(len(ts)-2, 0, -1):
    p = q.popleft()
    contval = np.polyval(p, paths[i])
    exerval = np.zeros(len(paths[i]))-alpha*ts[i]
    # identify the paths where we should exercise
    ind = exerval > contval
    payoff[ind] = exerval[ind]

np.mean(payoff)


3.707074125017274